# Out of Distribution Detection Examples and Strategies

## Detecting covariate shifts with Random Forests

One example from [Deep learning for coders with fastai & pytorch](https://www.amazon.com/Deep-Learning-Coders-fastai-PyTorch/dp/1492045527) is how to cleverly use a random forest to detect the possibility of having OOD data from covariate shifts.

Strategy:
1. Create your test/train split. Sometimes, especially for forecasing things in the future a good train/test split is NOT a random mix and division - instead, a training set may occur earlier in time, while the test occurs later in the future.  It is important to check if there are covariate shifts occuring as things change over time.
2. Use a RF to predict if a row comes from the training set or test set. If the model performs well, you have some detectable shift in your data - in this example, premised on shifts over time.
3. Examine the RF feature importances to explain what is changing over time.

Reasoning:
* RFs are fast, easy to train and are robust against most hyperparameter choices so your results won't be very dependent on selection of a bad model.
* Your data does need to be tabular and RF-friendly (i.e., hopefully no very high cardinality categorical features)